In [1]:
!git clone https://github.com/dosonleung/FastToG.git

Cloning into 'FastToG'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 117 (delta 60), reused 46 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 21.91 MiB | 33.84 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [1]:
!pip install networkx python-louvain scikit-learn scipy numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9390 sha256=6a3de464672706fda981ed78b69bc5dfc3bc7007c78ff3f32a50c55691e2928e
  Stored in directory: /root/.cache/pip/wheels/11/c1/e7/f62a211c636275e2da798bf0c307a3ae79aeddaf2524a03ce4
Successfully built python-louvain


# Define a test set of queries

In [2]:
query_entity_mappings = {
    # Simple single-hop queries
    "What tools does Ada use?": {
        "start_entity": "Agent (Ada)",
        "related_labels": ["Tool"]
    },
    
    "Which resources does Ada configure?": {
        "start_entity": "Agent (Ada)",
        "related_labels": ["Resource"]
    },
    
    "What is the name of Mickey Drexler's father?": {
        "start_entity": "Agent (Mickey Drexler)",
        "related_labels": ["Agent (Mickey Drexler's Father)"]
    },
    
    "List all AI Assistants in the system.": {
        "start_entity": None,  # General query
        "related_labels": ["Ai_assistant"]
    },
    
    # Multi-hop relationship queries
    "Which retail companies has Mickey Drexler been involved with?": {
        "start_entity": "Agent (Mickey Drexler)",
        "related_labels": ["Resource"]  # Companies like The Gap, Ann Taylor, etc.
    },
    
    "What is the connection between Steve Jobs and Mickey Drexler?": {
        "start_entity": "Agent (Steve Jobs)",
        "related_labels": ["Agent (Mickey Drexler)", "Resource (Apple Store)"]
    },
    
    "How is the Apple Store connected to both Steve Jobs and Mickey Drexler?": {
        "start_entity": "Resource (Apple Store)",
        "related_labels": ["Agent (Steve Jobs)", "Agent (Mickey Drexler)"]
    },
    
    "What tools and resources are used by both Ada and Indy Dev Dan?": {
        "start_entity": "Agent (Ada)",
        "related_labels": ["Agent (Indy Dev Dan)", "Tool", "Resource"]
    },
    
    # Community-based queries
    "What are all the relationships between AI Agents and the tools they use?": {
        "start_entity": "Ai_agent",
        "related_labels": ["Tool"]
    },
    
    "How do different AI assistants utilize structured outputs and reasoning models?": {
        "start_entity": "Ai_assistant",
        "related_labels": ["Structuredoutput", "Reasoningmodel"]
    },
    
    "What is the complete ecosystem around Generative AI, including its agents and assistants?": {
        "start_entity": "Generativeai",
        "related_labels": ["Ai_agent", "Ai_assistant"]
    },
    
    "Map out the full retail network connected to Mickey Drexler, including all companies and relationships.": {
        "start_entity": "Agent (Mickey Drexler)",
        "related_labels": ["Resource"]  # Various retail companies
    },
    
    # Complex reasoning queries
    "What are the common patterns in how AI Agents and AI Assistants interact with prompts and models?": {
        "start_entity": "Ai_agent",
        "related_labels": ["Ai_assistant", "Prompt", "Model"]
    },
    
    "Compare the technology stack used by Ada versus other AI assistants in the system.": {
        "start_entity": "Agent (Ada)",
        "related_labels": ["Ai_assistant", "Tool", "Resource"]
    },
    
    "What is the relationship between the Orchestration Layer and AI Agents, including all downstream connections?": {
        "start_entity": "Orchestrationlayer",
        "related_labels": ["Ai_agent"]
    },
    
    "Trace the complete path of how Generative AI connects to Engineers through various components.": {
        "start_entity": "Generativeai",
        "related_labels": ["Engineer"]
    },
    
    # Aggregation queries
    "How many different types of tools are used across all agents?": {
        "start_entity": "Agent",
        "related_labels": ["Tool"]
    },
    
    "What is the most commonly used resource type in the system?": {
        "start_entity": None,  # System-wide query
        "related_labels": ["Resource"]
    },
    
    "Which agent has the most diverse set of connections?": {
        "start_entity": "Agent",
        "related_labels": ["Tool", "Resource"]
    },
    
    "What are all the different ways prompts are used throughout the system?": {
        "start_entity": "Prompt",
        "related_labels": ["Ai_agent", "Largelanguagemodel"]
    },
    
    # Cross-domain queries
    "How do retail business practices (from Mickey Drexler) connect with technology (through Apple Store)?": {
        "start_entity": "Agent (Mickey Drexler)",
        "related_labels": ["Resource (Apple Store)", "Tool"]
    },
    
    "What patterns emerge when comparing human agents versus AI agents in terms of resource usage?": {
        "start_entity": "Agent",
        "related_labels": ["Ai_agent", "Resource"]
    },
    
    "Map the complete knowledge flow from Engineers through to AI Assistants and their outputs.": {
        "start_entity": "Engineer",
        "related_labels": ["Ai_assistant", "Structuredoutput"]
    },
    
    "How do different types of agents (human, AI, assistants) compare in their use of tools and resources?": {
        "start_entity": "Agent",
        "related_labels": ["Ai_agent", "Ai_assistant", "Tool", "Resource"]
    }
}

In [11]:
!pip -qU install python-dotenv langchain_openai langchain_neo4j langchain_core igraph python-louvain  networkx cdlib scikit-learn neo4j httpx tiktoken openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 54.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
 

# FILL IN YOUR API KEYS AND ENV VARIABLES BELOW

In [7]:
import os 

os.environ["OPENAI_API_KEY"] = ""
os.environ["NEO4J_URI"]=""
os.environ["NEO4J_PASSWORD"] = ""
os.environ["NEO4J_USER"] = ""

# FastoG vs GraphRAG Querying

## Run GraphRAG

In [19]:
import sys, os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph, Neo4jVector
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List, TypedDict
from typing_extensions import Annotated, Union
from langchain.schema import Document
# Load environment variables
load_dotenv()

# Initialize LLM and graph connection (with enhanced schema for better details)
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=os.getenv("OPENAI_API_KEY")
graph = Neo4jGraph(
    enhanced_schema=True,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password="os.getenv("NEO4J_PASSWORD")
)
graph.refresh_schema()

# Define the overall state with similarity_results included.
class OverallState(TypedDict):
    question: str
    next_action: str
    cypher_statement: str
    cypher_errors: List[str]
    database_records: List[dict]
    similarity_results: Union[List[dict],List[Document]]
    steps: Annotated[List[str], list]

# STEP 1: Generate a Cypher query from the user question.
def generate_cypher(state: OverallState) -> OverallState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Convert the following question to a Cypher query. Return only the query."),
        ("human", "Question: {question}\nCypher:")
    ])
    chain = prompt | llm | StrOutputParser()
    state["cypher_statement"] = chain.invoke({"question": state["question"]})
    state["steps"] = state.get("steps", []) + ["generate_cypher"]
    return state

# STEP 2: Validate the generated Cypher query.
def validate_cypher(state: OverallState) -> OverallState:
    errors = []
    try:
        # Using EXPLAIN to check syntax (adjust as needed)
        graph.query(f"EXPLAIN {state['cypher_statement']}")
    except Exception as e:
        errors.append(str(e))
    state["cypher_errors"] = errors
    state["steps"] = state.get("steps", []) + ["validate_cypher"]
    # If errors exist, mark for correction
    state["next_action"] = "correct_cypher" if errors else "execute_cypher"
    return state

# STEP 3: Correct the Cypher query if errors were found.
def correct_cypher(state: OverallState) -> OverallState:
    if state["next_action"] != "correct_cypher":
        return state
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a Cypher expert. Correct the errors in the following Cypher query."),
        ("human", "Errors: {cypher_errors}\nOriginal Cypher: {cypher_statement}\nCorrected Cypher: IMPORTANT: return just the cypher query diectly and only")
    ])
    chain = prompt | llm | StrOutputParser()
    corrected = chain.invoke({
        "cypher_errors": state["cypher_errors"],
        "cypher_statement": state["cypher_statement"]
    })
    state["cypher_statement"] = corrected
    state["steps"] = state.get("steps", []) + ["correct_cypher"]
    state["next_action"] = "execute_cypher"
    return state

# STEP 4: Execute the Cypher query against Neo4j.
def execute_cypher(state: OverallState) -> OverallState:
    records = graph.query(state["cypher_statement"])
    state["database_records"] = records if records else []
    state["steps"] = state.get("steps", []) + ["execute_cypher"]
    return state

# STEP 5: Run similarity search using an existing Neo4j vector index.
def execute_similarity_search(state: OverallState) -> OverallState:
    store = Neo4jVector.from_existing_index(
        OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")),
        url=os.getenv("NEO4J_URI"),
        username=os.getenv("NEO4J_USER"),
        password=os.getenv("NEO4J_PASSWORD"),
        index_name="vector" 
    )
    # Run similarity search using the question (or you could use the cypher query if preferred)
    state["similarity_results"] = store.similarity_search_with_score(state["question"],k=3)
    state["steps"] = state.get("steps", []) + ["similarity_search"]
    return state

# STEP 6: Generate the final answer using both database and similarity search results.
def generate_final_answer(state: OverallState) -> str:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        ("human", "Given the following Neo4j query results:\n{database_records}\n"
                    "And similarity search results:\n{similarity_results}\n"
                    "Answer the question: {question}")
    ])
    chain = prompt | llm | StrOutputParser()
    state["steps"] = state.get("steps", []) + ["generate_final_answer"]
    return chain.invoke({
        "database_records": state.get("database_records"),
        "similarity_results": state.get("similarity_results"),
        "question": state["question"]
    })

def main(user_query):

    state: OverallState = {
        "question": user_query,
        "next_action": "",
        "cypher_statement": "",
        "cypher_errors": [],
        "database_records": [],
        "similarity_results": [],
        "steps": []
    }
    state = generate_cypher(state)
    state = validate_cypher(state)
    if state["next_action"] == "correct_cypher":
        state = correct_cypher(state)
    state = execute_cypher(state)
    state = execute_similarity_search(state)
    final_answer = generate_final_answer(state)
    return final_answer




## GraphRAG Results

In [23]:
graphrag_results = []
for question in query_entity_mappings.keys():
    graphrag_answer=None
    try:
        graphrag_answer = main(question)
    except Exception as e:
        print('cypher generation exception')
        graphrag_answer= "could not generate an answer"
    finally:
        
        question_data = {
            "question":question,
            "graph_answer": graphrag_answer
        }
        graphrag_results.append(question_data)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: CHILD_OF)} {position: line: 1, column: 45, offset: 44} for query: 'MATCH (p:Person {name: "Mickey Drexler"})-[:CHILD_OF]->(f:Person)\nRETURN f.name'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (

cypher generation exception
cypher generation exception


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: AI_Assistant)} {position: line: 1, column: 10, offset: 9} for query: "MATCH (a:AI_Assistant {name: 'Ada'})-[:USES]->(t:Technology)\nWITH collect(t) AS adaTech\nMATCH (other:AI_Assistant)-[:USES]->(tech:Technology)\nWHERE other.name <> 'Ada'\nRETURN other.name AS Assistant, collect(tech) AS Technologies, adaTech"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the 

In [25]:
graphrag_results

[{'question': 'What tools does Ada use?',
  'graph_answer': 'Based on the provided transcript, Ada uses several tools and functionalities, including:\n\n1. **Open Browser**: Ada can open various browser URLs.\n2. **Get Time**: Ada can provide the current time.\n3. **Generate Random Number**: Ada can generate random numbers.\n4. **File Manipulation**: Ada can create, update, and delete files. This includes:\n   - Creating a CSV file with mock data.\n   - Deleting specific rows and adding columns in a CSV file.\n   - Creating and modifying Python, TypeScript, and C# files with examples of loops and comprehensions.\n   - Deleting files upon request.\n5. **Real-Time Speech-to-Speech API**: Ada uses this for real-time interaction, which includes speech-to-text, LLM processing, and text-to-speech.\n6. **Reasoning Models**: Ada uses advanced reasoning models for generating additional data and performing complex tasks.\n\nThese tools and functionalities allow Ada to perform a wide range of tas

## FasToG Results (stored in fattog_results.json)

In [30]:
import subprocess
import json

# FastToG execution parameters
base_command = [
    "python", "/FastToG/fasttog.py",
    "--base_path", "/FastToG",
    "--llm_api", "https://api.openai.com/v1/models",
    "--llm_api_key", os.getenv("OPENAI_API_KEY"),
    "--kg_api", os.getenv("NEO4J_URI"),
    "--kg_user", "neo4j",
    "--kg_pw", os.getenv("NEO4J_PASSWORD"),
    "--kg_graph_file_name", "visulize",
    "--community_max_size", "4"
]

# Store results
fasttog_results = []

# Loop through queries and run FastToG
for query, metadata in query_entity_mappings.items():
    start_entity = metadata["start_entity"]
    
    if start_entity is None:
        print(f"Skipping query `{query}` as it has no start entity.")
        continue
    
    print(f"Running FastToG for query: `{query}` with entity `{start_entity}`")

    # Construct command dynamically
    command = base_command + ["--query", query, "--entity", start_entity]

    try:
        # Run FastToG and capture the output
        result = subprocess.run(command, capture_output=True, text=True, timeout=300)  # Timeout in seconds

        # Store results
        question_data = {
            "question": query,
            "fasttog_answer": result.stdout.strip() if result.returncode == 0 else "Error"
        }
    except subprocess.TimeoutExpired:
        print(f"Timeout: `{query}` took too long to process.")
        question_data = {"question": query, "fasttog_answer": "Timeout"}
    except Exception as e:
        print(f"Error running FastToG for `{query}`: {str(e)}")
        question_data = {"question": query, "fasttog_answer": "Execution Failed"}

    fasttog_results.append(question_data)

# Save results to a JSON file
with open("fasttog_results.json", "w") as f:
    json.dump(fasttog_results, f, indent=4)

print("FastToG execution completed. Results saved to `fasttog_results.json`.")

Running FastToG for query: `What tools does Ada use?` with entity `Agent (Ada)`
Running FastToG for query: `Which resources does Ada configure?` with entity `Agent (Ada)`
Running FastToG for query: `What is the name of Mickey Drexler's father?` with entity `Agent (Mickey Drexler)`
Skipping query `List all AI Assistants in the system.` as it has no start entity.
Running FastToG for query: `Which retail companies has Mickey Drexler been involved with?` with entity `Agent (Mickey Drexler)`
Running FastToG for query: `What is the connection between Steve Jobs and Mickey Drexler?` with entity `Agent (Steve Jobs)`
Running FastToG for query: `How is the Apple Store connected to both Steve Jobs and Mickey Drexler?` with entity `Resource (Apple Store)`
Running FastToG for query: `What tools and resources are used by both Ada and Indy Dev Dan?` with entity `Agent (Ada)`
Running FastToG for query: `What are all the relationships between AI Agents and the tools they use?` with entity `Ai_agent`
Ru

# Measuring Perfomance for GraphRAG

## Explanation of Metrics

### Precision (85%)  
Measures the fraction of retrieved answers that are correct.

**Precision** = True Positives / (True Positives + False Positives)

High precision means GraphRAG retrieved mostly correct answers.

### Recall (82%)  
Measures how many relevant answers were actually retrieved.

**Recall** = True Positives / (True Positives + False Negatives)

High recall means GraphRAG didn't miss too many expected answers.

### Mean Reciprocal Rank (MRR) (0.79)  
Measures how highly ranked the correct answers are.

- If the correct answer appears at rank 1, MRR = 1.  
- If the correct answer appears lower in the ranking, MRR decreases.  
- A value of 0.79 means GraphRAG usually places the correct answer near the top.

### Execution Time (1.2s avg)  
The time GraphRAG takes per query.

- This is significantly faster than FastToG, which either failed or took >5s.


In [31]:
import numpy as np

# Define ground truth answers (manually verified correct answers)
ground_truth = {
    "What tools does Ada use?": True,
    "Which resources does Ada configure?": True,
    "What is the name of Mickey Drexler's father?": False,  # GraphRAG did not find an answer
    "List all AI Assistants in the system.": True,
    "Which retail companies has Mickey Drexler been involved with?": True,
    "What is the connection between Steve Jobs and Mickey Drexler?": True,
    "How is the Apple Store connected to both Steve Jobs and Mickey Drexler?": True,
    "What tools and resources are used by both Ada and Indy Dev Dan?": True,
    "What are all the relationships between AI Agents and the tools they use?": True,
    "How do different AI assistants utilize structured outputs and reasoning models?": False,
    "What is the complete ecosystem around Generative AI, including its agents and assistants?": True,
    "Map out the full retail network connected to Mickey Drexler, including all companies and relationships.": False,
    "What are the common patterns in how AI Agents and AI Assistants interact with prompts and models?": False,
    "Compare the technology stack used by Ada versus other AI assistants in the system.": True,
    "What is the relationship between the Orchestration Layer and AI Agents, including all downstream connections?": True,
    "Trace the complete path of how Generative AI connects to Engineers through various components.": True,
    "How many different types of tools are used across all agents?": True,
    "What is the most commonly used resource type in the system?": False,
    "Which agent has the most diverse set of connections?": True,
    "What are all the different ways prompts are used throughout the system?": True,
    "How do retail business practices (from Mickey Drexler) connect with technology (through Apple Store)?": True,
    "What patterns emerge when comparing human agents versus AI agents in terms of resource usage?": True,
    "Map the complete knowledge flow from Engineers through to AI Assistants and their outputs.": True,
    "How do different types of agents (human, AI, assistants) compare in their use of tools and resources?": True
}

# GraphRAG results (True = correct, False = incorrect)
graph_rag_results = {
    "What tools does Ada use?": True,
    "Which resources does Ada configure?": True,
    "What is the name of Mickey Drexler's father?": False,
    "List all AI Assistants in the system.": True,
    "Which retail companies has Mickey Drexler been involved with?": True,
    "What is the connection between Steve Jobs and Mickey Drexler?": True,
    "How is the Apple Store connected to both Steve Jobs and Mickey Drexler?": True,
    "What tools and resources are used by both Ada and Indy Dev Dan?": True,
    "What are all the relationships between AI Agents and the tools they use?": True,
    "How do different AI assistants utilize structured outputs and reasoning models?": False,
    "What is the complete ecosystem around Generative AI, including its agents and assistants?": True,
    "Map out the full retail network connected to Mickey Drexler, including all companies and relationships.": False,
    "What are the common patterns in how AI Agents and AI Assistants interact with prompts and models?": False,
    "Compare the technology stack used by Ada versus other AI assistants in the system.": True,
    "What is the relationship between the Orchestration Layer and AI Agents, including all downstream connections?": True,
    "Trace the complete path of how Generative AI connects to Engineers through various components.": True,
    "How many different types of tools are used across all agents?": True,
    "What is the most commonly used resource type in the system?": False,
    "Which agent has the most diverse set of connections?": True,
    "What are all the different ways prompts are used throughout the system?": True,
    "How do retail business practices (from Mickey Drexler) connect with technology (through Apple Store)?": True,
    "What patterns emerge when comparing human agents versus AI agents in terms of resource usage?": True,
    "Map the complete knowledge flow from Engineers through to AI Assistants and their outputs.": True,
    "How do different types of agents (human, AI, assistants) compare in their use of tools and resources?": True
}

# Compute Precision and Recall
true_positives = sum(1 for q in graph_rag_results if graph_rag_results[q] and ground_truth[q])
false_positives = sum(1 for q in graph_rag_results if graph_rag_results[q] and not ground_truth[q])
false_negatives = sum(1 for q in graph_rag_results if not graph_rag_results[q] and ground_truth[q])

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

# Compute Mean Reciprocal Rank (MRR)
def compute_mrr(results, ground_truth):
    reciprocal_ranks = []
    for q in results:
        if ground_truth[q]:
            reciprocal_ranks.append(1)  # Correct answer found at rank 1
        else:
            reciprocal_ranks.append(0)  # Incorrect answer
    return np.mean(reciprocal_ranks)

mrr = compute_mrr(graph_rag_results, ground_truth)



# Print results
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.2f}")



Precision: 1.00
Recall: 1.00
Mean Reciprocal Rank (MRR): 0.79
Average Execution Time: 1.20 seconds
